In [113]:
# import tensorflow as tf
#
# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#   try:
#     # Currently, memory growth needs to be the same across GPUs
#     for gpu in gpus:
#       tf.config.experimental.set_memory_growth(gpu, True)
#     logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#     print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
#   except RuntimeError as e:
#     # Memory growth must be set before GPUs have been initialized
#     print(e)

In [114]:
import tensorflow as tf
import keras
import gensim
import sklearn
import nltk
import konlpy
import pandas as pd
import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt
import re
import kss
import matplotlib.pyplot as plt
import pymssql
import pandas as pd
from matplotlib import font_manager
#%matplotlib inline
from wordcloud import WordCloud, STOPWORDS
from nltk.tokenize import sent_tokenize
from konlpy.tag import Okt
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
#tf-idf
from sklearn.decomposition import LatentDirichletAllocation
#topic modeling

from sqlalchemy import create_engine

In [115]:
pd.set_option("display.max_rows", 100)  # 출력할 최대 행 갯수를 100으로 설정
pd.set_option("display.max_columns", 500)  # 출력할 최대 열 갯수를 500개로 설정
pd.set_option("display.width", 500)  # 글자 수 기준 출력할 넓이 설정
pd.set_option("max_info_columns", 500)  # 열 기반 info가 주어질 경우, 최대 넓이

plt.style.use("seaborn-white")

user='root'
password='doslvkdlqm!'
host='192.168.1.116'
database='patent_analysis'

engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}/{database}', echo=True)
patent_naver = pd.read_sql("select * from tb_patent_info where company_name='NAVER' and main_claim is not null", con=engine)
patent_kakao = pd.read_sql("select * from tb_patent_info where company_name='KAKAO' and main_claim is not null", con=engine)

# 분석대상 칼럼 : "company_name", "applicant", "application_year", "title", "main_claim", "abstract"
# print(patent_naver)
# print(patent_kakao)

2021-07-07 14:38:46,698 INFO sqlalchemy.engine.Engine SHOW VARIABLES LIKE 'sql_mode'
2021-07-07 14:38:46,699 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-07-07 14:38:46,700 INFO sqlalchemy.engine.Engine SHOW VARIABLES LIKE 'lower_case_table_names'
2021-07-07 14:38:46,701 INFO sqlalchemy.engine.Engine [generated in 0.00053s] {}
2021-07-07 14:38:46,702 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2021-07-07 14:38:46,703 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-07-07 14:38:46,704 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2021-07-07 14:38:46,705 INFO sqlalchemy.engine.Engine [generated in 0.00047s] {'table_schema': 'patent_analysis', 'table_name': "select * from tb_patent_info where company_name='NAVER' and main_claim is not null"}
2021-07-07 14:38:46,706 INFO sqlalchemy.engine.Engine select * from tb_patent_info where company_name='NAVER' and main_claim is not null
2021-07

In [116]:
# 불용어처리
spwords = set(STOPWORDS)  # 기본적으로 많이 쓰는 단어 (제외할 단어)
spwords.add('상')  # 제외하고 싶은 단어 추가
spwords.add('그')
spwords.add('및')
spwords.add('위')
spwords.add('방법')
spwords.add('컴퓨터')
spwords.add('프로그램')
spwords.add('시스템')
spwords.add('장치')
spwords.add('상기')
spwords.add('인터넷')
spwords.add('사용자')
spwords.add('접속')
spwords.add('저장')
spwords.add('서비스')
stopwords =spwords



In [117]:
print(patent_naver['title'])
print("------------")

okt=Okt()
patent_naver['title'] = patent_naver.apply(lambda row: okt.nouns(row['title']), axis=1)
# print(patent_naver['title'])

patent_naver['title'] = patent_naver['title'].apply(lambda x: [word for word in x if word not in (stopwords)])
# print(patent_naver['title'])

# 역토큰화 (토큰화 작업을 되돌림)
tokenized_doc = patent_naver['title']
detokenized_doc = []
for i in range(len(patent_naver['title'])):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

patent_naver['title'] = detokenized_doc # 다시 data['REQ_CONTENT']에 재저장
print(patent_naver['title']) # 역토큰화 잘 되었는지 확인

# TF- IDF 행렬 만들기

vectorizer = TfidfVectorizer(stopwords, max_features=1000000) # 상위 1,000개의 단어를 보존
X = vectorizer.fit_transform(patent_naver['title'])
print(X.shape) # TF-IDF 행렬의 크기 확인

# 토픽 모델링
lda_model=LatentDirichletAllocation(n_components=10,learning_method='online',random_state=777,max_iter=1)
lda_top=lda_model.fit_transform(X)
print(lda_model.components_)
print(lda_model.components_.shape)

terms = vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장.

def get_topics(components, feature_names, n=10):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])
print("---- naver title topics: --------")
get_topics(lda_model.components_,terms)


0       증강 현실 기반 내비게이션 환경에서 객체의 그림자를 표시하기 위한 전자 장치 및 그...
1                                       동영상분류장치 및 동영상분류방법
2                         온라인 음악 서비스 제공 방법  장치 및 컴퓨터 프로그램
3           문자 자동인식이 불가한 3D 개체 기반 캡차 제공 방법  장치 및 컴퓨터 프로그램
4                             시각 정보 처리 기반의 위치 인식 방법 및 시스템
                              ...                        
1760                     온라인 상에서 직소퍼즐을 이용한 이벤트 진행 시스템 및방법
1761                      이동통신단말기에 광고를 표시하는 방법 및 그이동통신단말기
1762                                 관련 웹 사이트 제공 방법 및 시스템
1763                     웹 메일을 이용한 개인간 송금 서비스 제공장치 및 그 방법
1764                                      사이버머니를 이용한 경매방법
Name: title, Length: 1765, dtype: object
------------
0       증강 현실 기반 내비게이션 환경 객체 그림자 표시 전자 동작
1                           동영상 분류 동영상 분류
2                               온라인 음악 제공
3                    문자 자동 식이 개체 기반 캡차 제공
4                       시각 정보 처리 기반 위치 인식
                      ...                
1760                  온라인 직소 퍼즐 이용 이벤트 진행
1761          

/home/anyfive/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass input={'hence', 'then', '및', 'here', 'am', 'which', 'yourselves', 'their', 'all', '장치', 'having', "they're", 'because', 'i', 'do', "you'd", 'could', "i'm", 'get', 'r', 'should', '상기', 'under', 'this', 'by', 'over', "they'll", 'hers', 'myself', 'each', "what's", 'its', 'at', 'com', 'down', 'www', 'to', 'is', 'k', 'doing', 'theirs', 'himself', "wouldn't", 'that', 'are', "how's", 'in', 'our', "mustn't", 'but', 'had', 'did', 'no', "won't", "when's", '프로그램', 'whom', "we'll", 'when', 'yourself', 'so', 'both', 'themselves', 'otherwise', "who's", 'him', "she's", "shouldn't", 'other', 'from', "can't", 'has', 'it', 'an', "don't", 'same', 'you', 'on', 'was', "haven't", "aren't", 'also', "isn't", 'those', 'ours', "doesn't", "he's", 'not', "wasn't", 'them', 'where', '그', 'more', "you've", 'ought', 'there', 'been', 'just', "that's", 'these', 'http', 'like', 'any', 'what', 'herself', "hasn't", 'he', '

[[0.20327216 3.99709652 0.22018318 ... 0.20814348 1.2963654  0.67719063]
 [0.19559272 0.20805366 0.24186053 ... 0.20202853 1.90066883 0.20357533]
 [0.22645406 0.21959005 0.22850877 ... 0.21769642 0.2160951  0.21279792]
 ...
 [0.21295375 0.20739589 0.21002831 ... 0.59817648 0.19739401 0.20043185]
 [0.20785541 0.2098388  0.75647435 ... 0.20419477 0.21278205 0.21119762]
 [0.19762433 0.20647268 0.21861269 ... 0.19701382 0.19597621 0.19409916]]
(10, 1429)
---- naver title topics: --------
Topic 1: [('제공', 70.75), ('게임', 59.16), ('검색', 57.06), ('광고', 54.67), ('온라인', 44.86), ('컨텐츠', 26.68), ('이용', 26.58), ('정보', 26.38), ('통한', 21.06), ('네트워크', 20.49)]
Topic 2: [('기록', 25.74), ('관리', 21.36), ('매체', 20.84), ('제공', 17.28), ('이용', 16.55), ('검색', 15.76), ('판독', 15.36), ('문서', 13.67), ('메일', 13.44), ('게임', 13.15)]
Topic 3: [('커뮤니티', 15.53), ('결제', 9.33), ('과금', 7.81), ('지원', 5.97), ('입력', 5.53), ('전자', 5.24), ('엔진', 4.09), ('간의', 4.03), ('가로세로', 4.02), ('아이템', 3.77)]
Topic 4: [('관심', 4.72), ('영역', 

In [118]:
print(patent_kakao['title'])
print("------------")

okt=Okt()
patent_kakao['title'] = patent_kakao.apply(lambda row: okt.nouns(row['title']), axis=1)
# print(patent_kakao['title'])

patent_kakao['title'] = patent_kakao['title'].apply(lambda x: [word for word in x if word not in (stopwords)])
# print(patent_kakao['title'])

# 역토큰화 (토큰화 작업을 되돌림)
tokenized_doc = patent_kakao['title']
detokenized_doc = []
for i in range(len(patent_kakao['title'])):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

patent_kakao['title'] = detokenized_doc # 다시 data['REQ_CONTENT']에 재저장
print(patent_kakao['title']) # 역토큰화 잘 되었는지 확인

# TF- IDF 행렬 만들기

vectorizer = TfidfVectorizer(stopwords, max_features=1000000) # 상위 1,000개의 단어를 보존
X = vectorizer.fit_transform(patent_kakao['title'])
print(X.shape) # TF-IDF 행렬의 크기 확인

# 토픽 모델링
lda_model=LatentDirichletAllocation(n_components=10,learning_method='online',random_state=777,max_iter=1)
lda_top=lda_model.fit_transform(X)
print(lda_model.components_)
print(lda_model.components_.shape)

terms = vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장.

def get_topics(components, feature_names, n=10):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])
print("---- kakao title topics: --------")
get_topics(lda_model.components_,terms)




0                          메시지 서비스를 제공하는 방법  서버 및 사용자 단말
1                       동적 이모지를 표시하기 위한 단말의 동작 방법 및 그 단말
2                              애니메이션 말풍선을 표시하는 방법 및 그 단말
3      인스턴트 메시징 서비스에서 이모티콘을 제공하는 방법  이를 구현한 서버  사용자 단...
4      인스턴트 메시징 서비스에서 이모티콘을 제공하는 방법  이를 구현한 서버  사용자 단...
                             ...                        
739                                   인터넷을 이용한 금융 서비스 방법
740                                   인터넷 금융대출 서비스 중개 방법
741                                   인터넷 보험취급 서비스 중개 방법
742                                   인터넷 금융수신 서비스 중개 방법
743                                   인터넷 금융펀드 서비스 중개 방법
Name: title, Length: 744, dtype: object
------------
0                           메시지 제공 서버 단말
1                      적 이모지 표시 단말 동작 단말
2                        애니메이션 말풍선 표시 단말
3      인스턴트 메시 징 이모티콘 제공 구현 서버 단말 어플리케이션
4      인스턴트 메시 징 이모티콘 제공 구현 서버 단말 어플리케이션
                     ...                
739                                이용 금융
740                             금

In [119]:
print(patent_naver['main_claim'])
print("------------")

okt=Okt()
patent_naver['main_claim'] = patent_naver.apply(lambda row: okt.nouns(row['main_claim']), axis=1)
# print(patent_naver['main_claim'])

patent_naver['main_claim'] = patent_naver['main_claim'].apply(lambda x: [word for word in x if word not in (stopwords)])
# print(patent_naver['main_claim'])

# 역토큰화 (토큰화 작업을 되돌림)
tokenized_doc = patent_naver['main_claim']
detokenized_doc = []
for i in range(len(patent_naver['main_claim'])):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

patent_naver['main_claim'] = detokenized_doc # 다시 data['REQ_CONTENT']에 재저장
print(patent_naver['main_claim']) # 역토큰화 잘 되었는지 확인

# TF- IDF 행렬 만들기

vectorizer = TfidfVectorizer(stopwords, max_features=1000000) # 상위 1,000개의 단어를 보존
X = vectorizer.fit_transform(patent_naver['main_claim'])
print(X.shape) # TF-IDF 행렬의 크기 확인

# 토픽 모델링
lda_model=LatentDirichletAllocation(n_components=10,learning_method='online',random_state=777,max_iter=1)
lda_top=lda_model.fit_transform(X)
print(lda_model.components_)
print(lda_model.components_.shape)

terms = vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장.

def get_topics(components, feature_names, n=10):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])
print("---- naver claim topics: --------")
get_topics(lda_model.components_,terms)


0       전자 장치의 동작 방법에 있어서 상기 전자 장치의 위치를 기반으로  표시 모듈의 일...
1       슬라이드 쇼(slide show) 타입 동영상을 구별하는 동영상분류방법에 있어서  ...
2       사용자 단말의 사용자에게 음악 서비스를 제공하는 방법에 있어서 사용자 단말이 제1 ...
3       프로세서가  회전가능한 3D 개체를 생성하고  상기 3D 개체가 클라이언트 장치에서...
4       적어도 하나의 프로세서를 포함하는 컴퓨터 장치의 위치 인식 방법에 있어서 상기 적어...
                              ...                        
1760    사용자로부터 이벤트 참가신청을 수신하고 사용자에게 큐브를 제공하는 수단을 제공하는 ...
1761    이동통신단말기에 광고를 표시하는 방법에 있어서  상기 이동통신단말기에서 상기 이동통...
1762    (정정) 복수의 URL 북마크를 저장하고 있는 북마크 DB  표준 URL DB  관...
1763    송금자로부터 수금자의 전자우편 주소  송금액  송금자 결제정보를 포함한 제1 머니메...
1764    인터넷 웹사이트 상에서 제공되는 사이버머니를 이용하여 경매를 하는 방법에 있어서  ...
Name: main_claim, Length: 1765, dtype: object
------------
0       전자 동작 전자 위치 기반 표시 모듈 일부 영역 통해 표시 객체 검 출하 동작 객체...
1       슬라이드 쇼 타입 동영상 구별 동영상 분류 입력 대상 동영상 복수 프레임 추출 단계...
2       단말 음악 제공 단말 제 제 음악 컨텐츠 대한 제공 요청 입력 음악 컨텐츠 제공 요...
3       프로세서 회전 개체 생 개체 클라이언트 소정 방향 표시 개체 클라이언트 전송 클라이...
4       하나 프로세서 포함 위치 인식 하나 프로세서 카메라 통해 프레임 이미지 생 단계 하...
             

In [120]:
print(patent_kakao['main_claim'])
print("------------")

okt=Okt()
patent_kakao['main_claim'] = patent_kakao.apply(lambda row: okt.nouns(row['main_claim']), axis=1)
# print(patent_kakao['main_claim'])

patent_kakao['main_claim'] = patent_kakao['main_claim'].apply(lambda x: [word for word in x if word not in (stopwords)])
# print(patent_kakao['main_claim'])

# 역토큰화 (토큰화 작업을 되돌림)
tokenized_doc = patent_kakao['main_claim']
detokenized_doc = []
for i in range(len(patent_kakao['main_claim'])):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

patent_kakao['main_claim'] = detokenized_doc # 다시 data['REQ_CONTENT']에 재저장
print(patent_kakao['main_claim']) # 역토큰화 잘 되었는지 확인

# TF- IDF 행렬 만들기

vectorizer = TfidfVectorizer(stopwords, max_features=1000000) # 상위 1,000개의 단어를 보존
X = vectorizer.fit_transform(patent_kakao['main_claim'])
print(X.shape) # TF-IDF 행렬의 크기 확인

# 토픽 모델링
lda_model=LatentDirichletAllocation(n_components=10,learning_method='online',random_state=777,max_iter=1)
lda_top=lda_model.fit_transform(X)
print(lda_model.components_)
print(lda_model.components_.shape)

terms = vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장.

def get_topics(components, feature_names, n=10):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])
print("---- kakao claim topics: --------")
get_topics(lda_model.components_,terms)


0      메신저 서버에서 수행되는 메시지 서비스 제공 방법에 있어서 그룹 채팅을 위한 적어도...
1      인스턴트 메시징 서비스의 메시지가 적어도 하나의 동적 이모지- 상기 동적 이모지는 ...
2      인스턴트 메시징 서비스를 통하여  메시지를 수신하는 단계;상기 메시지가 텍스트 데이...
3      단말이 인스턴트 메시징 어플리케이션의 사용자 인터페이스 화면을 제공하는 방법으로서 ...
4      단말이 이모티콘을 제공하는 방법으로서 키워드별 매칭 텍스트들을 포함하는 키워드 딕셔...
                             ...                        
739    사용자 컴퓨터 시스템  가상은행 컴퓨터 시스템  금융기관 컴퓨터 시스템이 네트워크로...
740    인터넷 사용자가 인터넷에 접속하고  금융대출기관의 안내에 따라 금융대출 상품정보를 ...
741    인터넷 사용자가 인터넷에 접속하고  보험취급기관의 안내에 따라 보험가입에 대한 상품...
742    인터넷 사용자가 인터넷에 접속하고  금융수신기관의 안내에 따라 수신상품 정보를 취득...
743    인터넷 사용자가 인터넷에 접속하여 투자유치기관의 안내에 따라 투자상품 정보를 취득한...
Name: main_claim, Length: 744, dtype: object
------------
0      메신저 서버 수행 메시지 제공 그룹 채팅 하나 오픈 채팅 방 생 단계 그룹 채팅 접...
1      인스턴트 메시 징 메시지 하나 적 이모지 적 이모 정적 이모지 구별 시각 효과 인 ...
2      인스턴트 메시 징 통 메시지 수신 단계 메시지 텍스트 데이터 하나 적 이모지 적 이...
3      단말 인스턴트 메시 징 어플리케이션 인터페이스 화면 제공 텍스트 입력 창 이모티콘 ...
4      단말 이모티콘 제공 키 워드 별 매칭 텍스트 포함 키 워드 딕셔너리 단계 텍스트 입...
                             .